setting up a neural network regression using Keras with a dataset that is stored in a JSON file as a list of objects:

In [ ]:
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

# Load the data from the JSON file
with open('user_input/scenarios.json') as f:
    data = json.load(f)

# Extract the features and labels from the data
X = []
y = []
for example in data:
    X.append([
        example['weather'],
        example['vehicle'],
        example['traffic'],
        example['emergency'],
        example['timeOfDay'],
        example['location'],
        example['intersections'],
        example['pedestrians'],
        example['pedestrian_cross'],
        example['route_length'],

    ])
    y.append(example['total_difficulty_rating'])

# Load the data from the JSON file
with open('user_input/scenarios.json') as f:
    data = json.load(f)


# One-hot encode the features
from sklearn.preprocessing import OneHotEncoder

# Create a list of indices corresponding to the categorical columns in X
cat_cols = [0, 1, 2, 3, 4, 5]

# Initialize the one-hot encoder
encoder = OneHotEncoder(categories='auto', sparse=False)

# Fit the encoder on X
encoder.fit(X)

# Transform X using the encoder
X_encoded = encoder.transform(X)

# Print the shape of the encoded X
print(X_encoded.shape)

# Convert the data to NumPy arrays
X = X_encoded 
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(16, input_shape=(39,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

# Make predictions with the model
predictions = model.predict(X_test)


To generate new scenarios using the predictions from the neural network, you can use the predicted values as input for the parameters of the new scenarios. Here's an example of how you can do that:

Get the predicted values for each scenario from the neural network model.
Scale the predicted values back to their original range if you normalized the data during training.
Use the predicted values to generate new scenarios with higher difficulty scores. For example, you can increase the number of cars or pedestrians, or set the weather to a more challenging condition.
Save the new scenarios to a JSON file.

Here's an example of how you can generate new scenarios based on the predicted values:


In [ ]:
import json

predicted_values = predictions
# Scale the predicted values back to their original range if needed

# Generate new scenarios based on the predicted values
new_scenarios = []
for i in range(len(predicted_values)):
    # Increase the number of cars and pedestrians based on the predicted values
    num_cars = int(predicted_values[i][0] * 10) # Scale the value from 0-1 to 0-10
    pedestrians = True if predicted_values[i][1] > 0.5 else False # Set pedestrians to True if the value is above 0.5
    # Set other parameters to a fixed value or use the predicted values if appropriate
    scenario = {
        'weather': 'Rain',
        'road': 'City',
        'vehicle': 'Car',
        'traffic': 'Heavy',
        'emergency': 'No',
        'timeOfDay': 'Day',
        'location': 'Urban',
        "num_cars": num_cars,
        "intersection": True,
        "pedestrians": pedestrians,
        "pedestrian_cross": True,
    }
    new_scenarios.append(scenario)

# Save the new scenarios to a JSON file
with open('new_scenarios.json', 'w') as f:
    json.dump(new_scenarios, f, indent=4)


Enhanced version

In [ ]:

# Create a list of potential values for each feature
weather = ['Sunny', 'Rain', 'Thunderstorm']
vehicle = ['Small', 'Truck','Van']
traffic = ['Heavy', 'Light','Medium']
emergency = ['Yes', 'No']
timeOfDay = ['Day', 'Night','Dawn','Dusk']
location = ['Urban', 'Country','Downtown']
intersections = [0,1,2,3,4]
pedestrians = [True,False]
pedestrian_cross = [True, False]
route_length = [100,200,300]


# Generate three new scenarios with high difficulty scores
for i in range(3):
    # Randomly select values for each feature
    new_scenario = [
        np.random.choice(weather),
        np.random.choice(vehicle),
        np.random.choice(traffic),
        np.random.choice(timeOfDay),
        np.random.choice(location),
        np.random.choice(intersections),
        np.random.choice(route_length)
    ]
    
    # One-hot encode the new scenario
    new_scenario_encoded = encoder.transform([new_scenario])
    
    # Predict the difficulty rating for the new scenario
    difficulty_score = model.predict(new_scenario_encoded)[0][0]
    
    # Print the new scenario and its difficulty score
    print(f"New scenario {i+1}: {new_scenario} - Difficulty score: {difficulty_score:.2f}")